[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/klavis-ai/klavis/blob/main/examples/together-ai/Salesforce_Gmail_TogetherAI_Integration.ipynb)


# Together AI + Klavis Salesforce + Gmail Integration

# <img src="../../static/togetherai-klavis.png" width="700">

This tutorial demonstrates how to build a powerful AI agent using Together AI and Klavis MCP servers to:

- **Salesforce Integration**: Automatically find CRM data like opportunities
- **Gmail Integration**: Draft and send professional follow-up emails
- **AI-Powered**: Use Together AI's LLMs for intelligent email composition


## Prerequisites

Before we begin, you'll need:

- **Together AI API key** - Get yours at [together.ai](https://together.ai/)
- **Klavis AI API key** - Get yours at [klavis.ai](https://klavis.ai/)

In [1]:
# Install the required packages
%pip install -qU together klavis

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import json
import webbrowser
from together import Together
from klavis import Klavis
from klavis.types import McpServerName, ToolFormat

# Set environment variables
os.environ["TOGETHER_API_KEY"] = "YOUR_TOGETHER_API_KEY"  # Replace with your actual Together API key
os.environ["KLAVIS_API_KEY"] = "YOUR_KLAVIS_API_KEY"      # Replace with your actual Klavis API key

## Step 1: Initialize Clients and Create MCP Server Instances

First, let's set up our Together AI and Klavis clients, then create Salesforce and Gmail MCP server instances.

In [3]:
# Initialize clients
together_client = Together(api_key=os.getenv("TOGETHER_API_KEY"))
klavis_client = Klavis(api_key=os.getenv("KLAVIS_API_KEY"))

In [ ]:
# Create Salesforce MCP Server instance
salesforce_mcp_instance = klavis_client.mcp_server.create_server_instance(
    server_name=McpServerName.SALESFORCE,
    user_id="1234", 
    platform_name="TogetherAI" 
)

# Open OAuth URL for Salesforce authorization
webbrowser.open(salesforce_mcp_instance.oauth_url)
print(f"🔐 Opening OAuth authorization for Salesforce...")

🔐 Opening OAuth authorization for Salesforce...


In [ ]:
# Create Gmail MCP Server instance
gmail_mcp_instance = klavis_client.mcp_server.create_server_instance(
    server_name=McpServerName.GMAIL,
    user_id="1234",
    platform_name="TogetherAI"
)

# Open OAuth URL for Gmail authorization
webbrowser.open(gmail_mcp_instance.oauth_url)
print(f"🔐 Opening OAuth authorization for Gmail...")

🔐 Opening OAuth authorization for Gmail...


## Step 2: Create workflow 

Now we'll create a workflow that can work with both Salesforce and Gmail MCP servers.

In [17]:
class MultiServiceAgent:
    def __init__(self, together_client, klavis_client, mcp_server_urls, model="meta-llama/Llama-3.3-70B-Instruct-Turbo"):
        self.together = together_client
        self.klavis = klavis_client
        self.mcp_server_urls = mcp_server_urls
        self.model = model
    
    def process_request(self, user_message):
        # 1. Get available tools from all MCP servers and create tool-to-server mapping
        all_tools = []
        tool_to_server = {}  # Maps tool names to their server URLs
        
        for server_url in self.mcp_server_urls:
            mcp_tools = self.klavis.mcp_server.list_tools(
                server_url=server_url,
                format=ToolFormat.OPENAI,
            )
            all_tools.extend(mcp_tools.tools)
            
            for tool in mcp_tools.tools:
                tool_to_server[tool["function"]["name"]] = server_url
        
        # print all tools
        print(f"🔧 Available tools: {all_tools}")
        
        # 2. Initialize conversation
        messages = [
            {"role": "system", "content": "You are a helpful AI assistant with access to tools. Complete requested tasks step by step."},
            {"role": "user", "content": user_message}
        ]
        
        max_iterations = 10 
        iteration = 0
        
        # 3. Keep processing until no more tool calls are needed
        while iteration < max_iterations:
            iteration += 1
            print(f"🔄 Processing iteration {iteration}")
            
            # Call LLM with all available tools
            response = self.together.chat.completions.create(
                model=self.model,
                messages=messages,
                tools=all_tools
            )
            
            assistant_message = response.choices[0].message
            messages.append(assistant_message)
            
            # If tool calls are needed
            if assistant_message.tool_calls:
                
                # Execute tool calls
                for tool_call in assistant_message.tool_calls:
                    tool_name = tool_call.function.name
                    tool_args = json.loads(tool_call.function.arguments)
                    
                    print(f"🛠️ Calling tool: {tool_name} with args: {tool_args}")
                    
                    # Find the correct server for this tool
                    if tool_name in tool_to_server:
                        server_url = tool_to_server[tool_name]
                        
                        try:
                            tool_result = self.klavis.mcp_server.call_tools(
                                server_url=server_url,
                                tool_name=tool_name,
                                tool_args=tool_args,
                            )
                            print(f"✅ Tool {tool_name} executed successfully")
                        except Exception as e:
                            tool_result = f"Error executing tool {tool_name}: {str(e)}"
                            print(f"❌ Tool {tool_name} failed: {str(e)}")
                    else:
                        tool_result = f"Error: Tool {tool_name} not found in any server"
                        print(f"❌ Tool {tool_name} not found in any server")
                    
                    messages.append({
                        "role": "tool",
                        "tool_call_id": tool_call.id,
                        "content": str(tool_result)
                    })
                
                # Continue the loop to see if LLM wants to make more tool calls
                continue
            
            else:
                # No more tool calls needed, return the final response
                print(f"✅ Task completed in {iteration} iterations")
                return assistant_message.content
        
        # If we hit max iterations, return the last response
        print(f"⚠️ Reached max iterations ({max_iterations})")
        return assistant_message.content if assistant_message.content else "Task completed but reached iteration limit"

multi_agent = MultiServiceAgent(
    together_client=together_client,
    klavis_client=klavis_client,
    mcp_server_urls=[salesforce_mcp_instance.server_url, gmail_mcp_instance.server_url],
    model="meta-llama/Llama-3.3-70B-Instruct-Turbo"
)

# Single request that uses both services
multi_response = multi_agent.process_request(
    """Please do the following using the tools available:
    1. Find Together AI opportunities in Salesforce with next steps
    2. draft a follow-up email based on the opportunity next steps
    """
)

print(multi_response)


🔧 Available tools: [{'type': 'function', 'function': {'name': 'salesforce_get_accounts', 'description': 'Get accounts with flexible filtering options including name search, industry, and type.', 'parameters': {'type': 'object', 'properties': {'limit': {'type': 'integer', 'description': 'Maximum number of accounts to return (default: 50)', 'default': 50}, 'fields': {'type': 'array', 'items': {'type': 'string'}, 'description': 'Specific fields to retrieve'}, 'name_contains': {'type': 'string', 'description': 'Filter accounts by name containing this text (case-insensitive)'}, 'industry': {'type': 'string', 'description': 'Filter accounts by industry'}, 'account_type': {'type': 'string', 'description': 'Filter accounts by type'}}, 'required': []}}}, {'type': 'function', 'function': {'name': 'salesforce_create_account', 'description': 'Create a new account in Salesforce.', 'parameters': {'type': 'object', 'properties': {'account_data': {'type': 'object', 'description': 'Account data includi

## Summary

This tutorial demonstrated how to create a powerful Salesforce + Gmail integration using Together AI and Klavis MCP servers.

### 🚀 **Key Features:**

- **Simple Agent Classes**: Easy-to-use Agent classes that work with any MCP server
- **Flexible Model Selection**: Support for various Together AI models (Llama, Qwen, etc.)
- **Real-time Execution**: Direct tool execution through Klavis API


### 🔧 **Next Steps:**

- **Try More MCP Servers**: Integrate additional MCP servers like Slack, Notion, or Linear
- **Custom Workflows**: Create more sophisticated multi-step workflows
- **Error Handling**: Add robust error handling and retry logic
- **Production Deployment**: Scale for production use with proper monitoring

**Happy building with Together AI and Klavis!** 🚀📊📧
